In [13]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import json
import numpy as np
headers = {
    "user-agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36",
    'Connection':'close'
}

# 编写Crawl（）类

## 筛选

In [7]:
js_keys = [
    'id', 'photo', 'name', 'blurb', 'goal', 'pledged', 'state', 'slug',
    'disable_communication', 'country', 'country_displayable_name', 'currency',
    'currency_symbol', 'currency_trailing_code', 'deadline',
    'state_changed_at', 'created_at', 'launched_at', 'staff_pick',
    'is_starrable', 'backers_count', 'static_usd_rate', 'usd_pledged',
    'converted_pledged_amount', 'fx_rate', 'usd_exchange_rate',
    'current_currency', 'usd_type', 'creator', 'location', 'category',
    'profile', 'spotlight', 'urls', 'percent_funded', 'is_liked', 'is_disliked'
]

In [8]:
class Crawler():
    def __init__(self):
        self.data_ls=[]#用于存储抓取的数据，每个元素都是一个项目的信息
        self.page_html=None#用于承载page=i的网页源代码
        self.error_ls=[]#用于存储提取目标信息失败的网页源代码【实践发现，存储的意义微小，所以可以无视它】
    def read(self,location):
        '''
        按照最初的两步走分工，一人负责下载page=i（i=1,…,165）网页源代码，并保存到本地,收纳为一个文件夹
        所以，read方法旨在读取另一位同学已经下载好的源代码文件，起承上启下之效
        '''
        with open(location,'r',encoding='utf-8')as f:
            self.page_html=f.read()
            print(location)
    def select(self):
        '''
        selec旨在提取目标信息
        目标信息是json格式
        
        '''
        soup = BeautifulSoup(self.page_html,'html.parser') # soup解析源代码，
        #一个page有12个项目信息
        div_ls = soup.find_all("div",class_='js-react-proj-card grid-col-12 grid-col-6-sm grid-col-4-lg')
        #用soup的find_all 所以理应提取12个json
        pattern='{"id"[^>]+}'#正则表达式提取目标json数据
        
        #遍历每个page的项目信息
        for i in range(len(div_ls)):
            div=div_ls[i]
            print(i)#打印是该page的第几个项目
            try:#可能因为没搜索到，用try语句避免AttributeError
                js=re.search(pattern,str(div)).group(0)#正则表达式搜索
                js_dict=json.loads(js)#将搜索到的json数据转化为字典
#                 print(js_dict['id'], '\n',js_dict['name'],'\n', js_dict['blurb'])
                self.data_ls.append([js_dict[key] for key in js_keys])#使用列表推导式，将字典的每个值打包成一个tuple，塞进列表self.data_ls
            
            #正则表达式没有检索到目标数据，会引发报错
            except AttributeError:#故写此错误捕捉语句
                print('AttributeError')
#                 print(div)
                self.error_ls.append(div)
    def main(self,path=None):
        '''
        遍历文件夹里的每个储存page源代码的文件，并进行read和selec操作
        ——可惜没有用到QAQ
        '''
        for loc in os.listdir(path):
            print(loc)
            self.read(loc)
            self.select()
        print('Successfully iterated.')
    def dataframe(self):
        '''
        以pandas的DataFrame数据类型，展现以抓取的数据
        '''
        return pd.DataFrame(np.array(self.data_ls),columns=js_keys)
    def store(self,path='./data_sotred.xlsx'):
        '''
        将抓取到的数据存为excel到本地，默认名为data_sotred.xlsx
        '''
        pd.DataFrame(np.array(self.data_ls),columns=js_keys).to_excel(path,index=False)
        print("Successfully stored.")

## 抓取实践

### crawl_go

In [11]:
crawl_go=Crawler()#创建实例crawl_go
for i in range(1,20):#抓取前20个page
    url='https://www.kickstarter.com/discover/advanced?category_id=16&woe_id=24865671&sort=magic&seed=2722523'    
    url+="&page={}".format(i)
    print('-*-'*30)
    print(i)
    crawl_go.page_html=requests.get(url,headers=headers).text
    crawl_go.select()
    time.sleep(3)

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
1
0
1
2
3
4
5
6
7
8
9
10
AttributeError
11
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
2
0
AttributeError
1
2
3
4
5
6
7
8
9
AttributeError
10
11
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
3
0
1
2
3
4
5
6
7
8
9
10
11
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
4
0
1
2
3
4
5
6
7
8
9
10
11
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
5
0
AttributeError
1
AttributeError
2
3
4
5
6
7
8
9
10
11
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
6
0
1
2
AttributeError
3
4
5
AttributeError
6
7
8
9
10
11
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
7
0
1
2
3
4
5
6
7
8
9
10
11
AttributeError
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*

In [14]:
#21时被打断
for i in range(21,25):#抓取前20个page
    url='https://www.kickstarter.com/discover/advanced?category_id=16&woe_id=24865671&sort=magic&seed=2722523'    
    url+="&page={}".format(i)
    print('-*-'*30)
    print(i)
    crawl_go.page_html=requests.get(url,headers=headers).text
    crawl_go.select()
    time.sleep(3)

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
21
0
1
2
3
4
5
6
7
8
9
10
11
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
22
0
1
2
3
4
5
6
7
8
9
10
AttributeError
11
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
23
0
1
2
3
4
5
AttributeError
6
7
8
9
AttributeError
10
11
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
24
0
1
2
3
AttributeError
4
5
6
7
AttributeError
8
9
10
11


In [15]:
#21时被打断
for i in range(25,40):#抓取前20个page
    url='https://www.kickstarter.com/discover/advanced?category_id=16&woe_id=24865671&sort=magic&seed=2722523'    
    url+="&page={}".format(i)
    print('-*-'*30)
    print(i)
    crawl_go.page_html=requests.get(url,headers=headers).text
    crawl_go.select()
    time.sleep(3)

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
25
0
1
2
3
4
5
6
7
8
9
10
AttributeError
11
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
26
0
1
2
3
4
5
6
AttributeError
7
8
9
AttributeError
10
11
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
27
0
1
2
3
AttributeError
4
AttributeError
5
6
AttributeError
7
8
9
10
11
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
28
0
1
2
3
4
5
6
AttributeError
7
8
9
10
11
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
29
0
1
2
3
AttributeError
4
5
6
7
8
9
10
11
AttributeError
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
30
0
AttributeError
1
2
3
AttributeError
4
5
AttributeError
6
7
8
AttributeError
9
AttributeError
10
11
AttributeError
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--

In [16]:
for i in range(40,80):#抓取前20个page
    url='https://www.kickstarter.com/discover/advanced?category_id=16&woe_id=24865671&sort=magic&seed=2722523'    
    url+="&page={}".format(i)
    print('-*-'*30)
    print(i)
    crawl_go.page_html=requests.get(url,headers=headers).text
    crawl_go.select()
    time.sleep(3)

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
40
0
AttributeError
1
2
3
4
5
6
7
8
9
10
11
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
41
0
1
2
3
4
5
AttributeError
6
AttributeError
7
8
9
10
11
AttributeError
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
42
0
AttributeError
1
AttributeError
2
3
4
5
AttributeError
6
7
8
9
10
11
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
43
0
1
AttributeError
2
3
4
5
6
7
AttributeError
8
9
10
11
AttributeError
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
44
0
AttributeError
1
2
3
4
5
6
AttributeError
7
8
9
10
11
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
45
0
AttributeError
1
2
3
4
5
6
7
8
9
10
11
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--

In [18]:
crawl_go.dataframe().to_excel('page小于80.xlsx',index=False)

In [143]:
import time
time.sleep(1)

In [144]:
for i in range(50,100):
    url='https://www.kickstarter.com/discover/advanced?category_id=16&woe_id=24865671&sort=magic&seed=2722523'    
    url+="&page={}".format(i)
    print('-*-'*50)
    print(i)
    crawl_go.page_html=requests.get(url,headers=headers).text
    crawl_go.select()
    time.sleep(3)    

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
50
0
1
2
3
4
5
6
7
8
9
10
11
AttributeError
<div class="js-react-proj-card grid-col-12 grid-col-6-sm grid-col-4-lg" data-pid="2124676256" data-project="{&quot;id&quot;:2124676256,&quot;photo&quot;:{&quot;key&quot;:&quot;assets/014/535/891/6177ebbb02596dae5dc6d7ebaf54273d_original.png&quot;,&quot;full&quot;:&quot;https://ksr-ugc.imgix.net/assets/014/535/891/6177ebbb02596dae5dc6d7ebaf54273d_original.png?ixlib=rb-4.0.2&amp;crop=faces&amp;w=560&amp;h=315&amp;fit=crop&amp;v=1483175609&amp;auto=format&amp;frame=1&amp;q=92&amp;s=cc1f37de81c04c1d149ab23bf601e6f8&quot;,&quot;ed&quot;:&quot;https://ksr-ugc.imgix.net/assets/014/535/891/6177ebbb02596dae5dc6d7ebaf54273d_original.png?ixlib=rb-4.0.2&amp;crop=faces&amp;w=352&amp;h=198&amp;fit=crop&amp;v=1483175609&amp;auto=format&amp;frame=1&amp;q=92&amp;s=d6ca0b4225d7be7e477f6e9dde5c65ae&quot;,&quot;me

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
51
0
1
2
3
4
5
6
AttributeError
<div class="js-react-proj-card grid-col-12 grid-col-6-sm grid-col-4-lg" data-pid="88057035" data-project="{&quot;id&quot;:88057035,&quot;photo&quot;:{&quot;key&quot;:&quot;assets/023/030/825/f18dccbc8b02a2b94ea9fa577d65a843_original.png&quot;,&quot;full&quot;:&quot;https://ksr-ugc.imgix.net/assets/023/030/825/f18dccbc8b02a2b94ea9fa577d65a843_original.png?ixlib=rb-4.0.2&amp;crop=faces&amp;w=560&amp;h=315&amp;fit=crop&amp;v=1540834572&amp;auto=format&amp;frame=1&amp;q=92&amp;s=c5f4ff4f9bad961f4e953cd55a6902f8&quot;,&quot;ed&quot;:&quot;https://ksr-ugc.imgix.net/assets/023/030/825/f18dccbc8b02a2b94ea9fa577d65a843_original.png?ixlib=rb-4.0.2&amp;crop=faces&amp;w=352&amp;h=198&amp;fit=crop&amp;v=1540834572&amp;auto=format&amp;frame=1&amp;q=92&amp;s=e050c0d25110f315919f2836697035a4&quot;,&quot;med&quot;:&quot;ht

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
52
0
1
2
3
4
AttributeError
<div class="js-react-proj-card grid-col-12 grid-col-6-sm grid-col-4-lg" data-pid="1508858303" data-project="{&quot;id&quot;:1508858303,&quot;photo&quot;:{&quot;key&quot;:&quot;assets/030/490/053/e1d44a13f0978fd95085b3f84219a56f_original.jpg&quot;,&quot;full&quot;:&quot;https://ksr-ugc.imgix.net/assets/030/490/053/e1d44a13f0978fd95085b3f84219a56f_original.jpg?ixlib=rb-4.0.2&amp;crop=faces&amp;w=560&amp;h=315&amp;fit=crop&amp;v=1599359354&amp;auto=format&amp;frame=1&amp;q=92&amp;s=06cfdac43fa5785130302f6d4149380d&quot;,&quot;ed&quot;:&quot;https://ksr-ugc.imgix.net/assets/030/490/053/e1d44a13f0978fd95085b3f84219a56f_original.jpg?ixlib=rb-4.0.2&amp;crop=faces&amp;w=352&amp;h=198&amp;fit=crop&amp;v=1599359354&amp;auto=format&amp;frame=1&amp;q=92&amp;s=df68f3e38ae01304cdc9c80a342c1361&quot;,&quot;med&quot;:&quot;ht

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
53
0
AttributeError
<div class="js-react-proj-card grid-col-12 grid-col-6-sm grid-col-4-lg" data-pid="1882239619" data-project="{&quot;id&quot;:1882239619,&quot;photo&quot;:{&quot;key&quot;:&quot;assets/031/365/826/1b0e133c9a873e7384a6af4ec0cdf109_original.jpg&quot;,&quot;full&quot;:&quot;https://ksr-ugc.imgix.net/assets/031/365/826/1b0e133c9a873e7384a6af4ec0cdf109_original.jpg?ixlib=rb-4.0.2&amp;crop=faces&amp;w=560&amp;h=315&amp;fit=crop&amp;v=1605062891&amp;auto=format&amp;frame=1&amp;q=92&amp;s=f9b9f619c70de224c3f05cbc13cc0d62&quot;,&quot;ed&quot;:&quot;https://ksr-ugc.imgix.net/assets/031/365/826/1b0e133c9a873e7384a6af4ec0cdf109_original.jpg?ixlib=rb-4.0.2&amp;crop=faces&amp;w=352&amp;h=198&amp;fit=crop&amp;v=1605062891&amp;auto=format&amp;frame=1&amp;q=92&amp;s=fb38b786361ae674769871e908c0271f&quot;,&quot;med&quot;:&quot;https://ks

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
54
0
1
AttributeError
<div class="js-react-proj-card grid-col-12 grid-col-6-sm grid-col-4-lg" data-pid="622294718" data-project="{&quot;id&quot;:622294718,&quot;photo&quot;:{&quot;key&quot;:&quot;assets/027/802/525/984fbb4d6efef5ca0b1fdb3329e699f8_original.jpg&quot;,&quot;full&quot;:&quot;https://ksr-ugc.imgix.net/assets/027/802/525/984fbb4d6efef5ca0b1fdb3329e699f8_original.jpg?ixlib=rb-4.0.2&amp;crop=faces&amp;w=560&amp;h=315&amp;fit=crop&amp;v=1579484939&amp;auto=format&amp;frame=1&amp;q=92&amp;s=f2a1ff7a0d8c2093e8eadab41ee782c9&quot;,&quot;ed&quot;:&quot;https://ksr-ugc.imgix.net/assets/027/802/525/984fbb4d6efef5ca0b1fdb3329e699f8_original.jpg?ixlib=rb-4.0.2&amp;crop=faces&amp;w=352&amp;h=198&amp;fit=crop&amp;v=1579484939&amp;auto=format&amp;frame=1&amp;q=92&amp;s=bbaed97dba6b27ced0e0a4c6d71a413f&quot;,&quot;med&quot;:&quot;https://ks

...

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
94
0
1
AttributeError
<div class="js-react-proj-card grid-col-12 grid-col-6-sm grid-col-4-lg" data-pid="598119731" data-project="{&quot;id&quot;:598119731,&quot;photo&quot;:{&quot;key&quot;:&quot;assets/026/110/579/ca8b053475483d7ec221c498d97628dd_original.jpg&quot;,&quot;full&quot;:&quot;https://ksr-ugc.imgix.net/assets/026/110/579/ca8b053475483d7ec221c498d97628dd_original.jpg?ixlib=rb-4.0.2&amp;crop=faces&amp;w=560&amp;h=315&amp;fit=crop&amp;v=1565587035&amp;auto=format&amp;frame=1&amp;q=92&amp;s=0faf9195f01d74bad0af4136efce9415&quot;,&quot;ed&quot;:&quot;https://ksr-ugc.imgix.net/assets/026/110/579/ca8b053475483d7ec221c498d97628dd_original.jpg?ixlib=rb-4.0.2&amp;crop=faces&amp;w=352&amp;h=198&amp;fit=crop&amp;v=1565587035&amp;auto=format&amp;frame=1&amp;q=92&amp;s=f0f12c1b19d086390ab4e3219488f544&quot;,&quot;med&quot;:&quot;https://ks

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
95
0
AttributeError
<div class="js-react-proj-card grid-col-12 grid-col-6-sm grid-col-4-lg" data-pid="2131305627" data-project="{&quot;id&quot;:2131305627,&quot;photo&quot;:{&quot;key&quot;:&quot;assets/017/279/391/294fcc823dd046040781011d1d87de43_original.png&quot;,&quot;full&quot;:&quot;https://ksr-ugc.imgix.net/assets/017/279/391/294fcc823dd046040781011d1d87de43_original.png?ixlib=rb-4.0.2&amp;crop=faces&amp;w=560&amp;h=315&amp;fit=crop&amp;v=1498575846&amp;auto=format&amp;frame=1&amp;q=92&amp;s=b45e60aff574c3e73e33ece8dc47987a&quot;,&quot;ed&quot;:&quot;https://ksr-ugc.imgix.net/assets/017/279/391/294fcc823dd046040781011d1d87de43_original.png?ixlib=rb-4.0.2&amp;crop=faces&amp;w=352&amp;h=198&amp;fit=crop&amp;v=1498575846&amp;auto=format&amp;frame=1&amp;q=92&amp;s=487ae682f5ffdd26413bf7362b5b67ae&quot;,&quot;med&quot;:&quot;https://ks

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
96
0
1
2
AttributeError
<div class="js-react-proj-card grid-col-12 grid-col-6-sm grid-col-4-lg" data-pid="1919702176" data-project="{&quot;id&quot;:1919702176,&quot;photo&quot;:{&quot;key&quot;:&quot;assets/023/982/952/4ddc022a8573d0ab9a7c174377ac0d05_original.jpg&quot;,&quot;full&quot;:&quot;https://ksr-ugc.imgix.net/assets/023/982/952/4ddc022a8573d0ab9a7c174377ac0d05_original.jpg?ixlib=rb-4.0.2&amp;crop=faces&amp;w=560&amp;h=315&amp;fit=crop&amp;v=1549260740&amp;auto=format&amp;frame=1&amp;q=92&amp;s=231218c9b2331f8e17d22c7080a25d46&quot;,&quot;ed&quot;:&quot;https://ksr-ugc.imgix.net/assets/023/982/952/4ddc022a8573d0ab9a7c174377ac0d05_original.jpg?ixlib=rb-4.0.2&amp;crop=faces&amp;w=352&amp;h=198&amp;fit=crop&amp;v=1549260740&amp;auto=format&amp;frame=1&amp;q=92&amp;s=cc8bbd67c135335c7bc9b7bcbb8aa29c&quot;,&quot;med&quot;:&quot;https:

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
97
0
1
2
3
4
5
6
7
8
9
10
11
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
98
0
1
2
3
AttributeError
<div class="js-react-proj-card grid-col-12 grid-col-6-sm grid-col-4-lg" data-pid="626621154" data-project="{&quot;id&quot;:626621154,&quot;photo&quot;:{&quot;key&quot;:&quot;assets/012/267/263/825f0914c827258f76565f606dc6a331_original.jpg&quot;,&quot;full&quot;:&quot;https://ksr-ugc.imgix.net/assets/012/267/263/825f0914c827258f76565f606dc6a331_original.jpg?ixlib=rb-4.0.2&amp;crop=faces&amp;w=560&amp;h=315&amp;fit=crop&amp;v=1463748530&amp;auto=format&amp;frame=1&amp;q=92&amp;s=f8315fd511ec6714a73af1867d4791e7&quot;,&quot;ed&quot;:&quot;https://ksr-ugc.imgix.net/assets/012/267/263/825f0914c827258f76565f606dc6a331_original.jpg?ixlib=rb-4

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
99
0
AttributeError
<div class="js-react-proj-card grid-col-12 grid-col-6-sm grid-col-4-lg" data-pid="1043204069" data-project="{&quot;id&quot;:1043204069,&quot;photo&quot;:{&quot;key&quot;:&quot;assets/014/326/116/4c571bdcfa4ae6e59abd220f35775113_original.jpg&quot;,&quot;full&quot;:&quot;https://ksr-ugc.imgix.net/assets/014/326/116/4c571bdcfa4ae6e59abd220f35775113_original.jpg?ixlib=rb-4.0.2&amp;crop=faces&amp;w=560&amp;h=315&amp;fit=crop&amp;v=1479216688&amp;auto=format&amp;frame=1&amp;q=92&amp;s=d19b230e13ad9717b4c991db111001cd&quot;,&quot;ed&quot;:&quot;https://ksr-ugc.imgix.net/assets/014/326/116/4c571bdcfa4ae6e59abd220f35775113_original.jpg?ixlib=rb-4.0.2&amp;crop=faces&amp;w=352&amp;h=198&amp;fit=crop&amp;v=1479216688&amp;auto=format&amp;frame=1&amp;q=92&amp;s=83f36c8b4b9762c33af140af7de72d2c&quot;,&quot;med&quot;:&quot;https://ks

In [152]:
len(crawl_go.data_ls),len(crawl_go.error_ls),len(crawl_go.error_ls)/len(crawl_go.data_ls)

(1007, 181, 0.17974180734856007)

# 编前摸索_ attribute error

In [133]:
import re
import json

div_ls = soup.find_all("div",class_='js-react-proj-card grid-col-12 grid-col-6-sm grid-col-4-lg') # 提取符合条件的第一个元素

print(len(div_ls))
pat1='{"id"[^>]+}'
data_ls=[]
for i in range(len(div_ls)):
#     print(div)
    div=div_ls[i]
    print(i)
    try:
        js=re.search(pat1,str(div)).group(0)
        print(js)
        js_dict=json.loads(js)
#         print(js_dict)    
#         print(js_dict['id'], '\n',js_dict['name'],'\n', js_dict['blurb'])
        data_ls.append([js_dict[key] for key in js_keys])
    except AttributeError:
        print('AttributeError')

12
0
{"id":1112997123,"photo":{"key":"assets/034/953/715/99e753456d7ff6619a52167d8f138850_original.png","full":"https://ksr-ugc.imgix.net/assets/034/953/715/99e753456d7ff6619a52167d8f138850_original.png?ixlib=rb-4.0.2&amp;crop=faces&amp;w=560&amp;h=315&amp;fit=crop&amp;v=1632223358&amp;auto=format&amp;frame=1&amp;q=92&amp;s=ddb3e7ea643404af61193797f3e21a23","ed":"https://ksr-ugc.imgix.net/assets/034/953/715/99e753456d7ff6619a52167d8f138850_original.png?ixlib=rb-4.0.2&amp;crop=faces&amp;w=352&amp;h=198&amp;fit=crop&amp;v=1632223358&amp;auto=format&amp;frame=1&amp;q=92&amp;s=000b3214d6b18b49a91bcf36bd110c46","med":"https://ksr-ugc.imgix.net/assets/034/953/715/99e753456d7ff6619a52167d8f138850_original.png?ixlib=rb-4.0.2&amp;crop=faces&amp;w=272&amp;h=153&amp;fit=crop&amp;v=1632223358&amp;auto=format&amp;frame=1&amp;q=92&amp;s=9f5bac34bb6b8401c57f4f2d527215e0","little":"https://ksr-ugc.imgix.net/assets/034/953/715/99e753456d7ff6619a52167d8f138850_original.png?ixlib=rb-4.0.2&amp;crop=faces&

In [202]:
zhihu=requests.get('https://zhuanlan.zhihu.com/p/33483400',headers=headers)
zhihu=BeautifulSoup(zhihu.text, 'html.parser')

In [205]:
for p in zhihu.find_all('p'):
    print(p.text)

在学术研究中，清晰明确是第一要义。但在博士生的辅导当中，经常在一张嘴就卡住。
举例一个对话。

你做的是什么研究？--“关于企业的可持续发展。”
酷。具体是是什么研究问题？---“企业如何做到可持续发展。”
那…… 具体你想研究那个方面，从什么角度？---“就是，企业如何保持长期持续发展。”
我是说，你的研究问题是？research question？是个问题，是个问句。
---“哦，问句。企业怎样保持长期可持续发展？”
这是非常熟悉和频繁的对话，尤其在博士生的第一年里。这里有两个概念不清。
其实是三个。
研究领域，研究课题和研究问题。Research area (or focus), Research Topic andresearch question.
研究领域很好懂，是指你研究的东西属于哪个圈子。比如上面的“企业可持续发展”。比如，“消费者行为”，比如“犯罪行为”。这是给对方一个初步印象，毕竟研究这回事包括世间万物。
研究课题，就是researchtopic, 是继续细化领域。指向这个大话题里的哪一块儿。是研究个什么“事儿”。顺着上面的例子，比如，
“环境、经济和社会与企业可持续发展的关系”。
比如，“中国消费者的网络购物行为”。
比如，“经济不平衡与犯罪率的关系”。
这还不够。
要说明你的研究课题，你就开始细化你的研究对象，你开始描述具体的角度，你要开始解释在什么语境里、什么角度、谁作用了谁。这里就出现了多于一个的主体。描述故事我们有“时间，地点，人物”。做研究同样。你要描述的是，
“在目前经济与商业模式快速发展的背景下（时间），在中国，在传统制造行业（地点），如何根据供给侧改革核心要素（人物）调整新市场开拓战略（人物）”；
“在当下市场需求多样化的趋势下（时间），传统制造行业在质控外包的产供链三方结构里(地点)，如何通过质控外包方（人物）保证质量管理文化（人物）的完整性以及促进再造”
“在中国（地点）目前年轻一代更倾向网络购物的趋势下（时间），以往消费者的评论(人物)如何影响潜在消费者的购买决策（人物）”。
这是你的研究课题。这里你要简洁说清，在哪儿，谁影响了谁。这里一定是多余一个作用力（人物），否则没有剧情—如果没有变化，何需研究。当然“时间”一项我们一般都是研究当下，所以很多时候不专门写出来，但是不代表没有。因为很多领域的研究都是聚焦以往，比如罗马时